In [3]:
# coding : utf-8
'''
@creat_time = 2022/6/11,19:33
@auther = MrCrimson
Email : mrcrimson@163.com
'''
import cv2
import random,time,hashlib,itertools
import numpy as np

visible_text = "MrCrimson"
ID = 123456789
begin_time = time.ctime()

In [4]:
#embed watermark
def Embed_watermark(video_root):
    cap = cv2.VideoCapture(video_root)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc('D','I','V','X')
    out = cv2.VideoWriter("embed_video.avi",fourcc,fps,(w,h))
    while cap.isOpened():
        ret,img = cap.read()
        if ret == True:
            embed_img,size = img_embed_watermark_dct(img)
            frame = embed_img
            out.write(frame)
        else:
            break
    cap.release()
    out.release()
    return size

def Extract_watermark(video_root,size):
    cap = cv2.VideoCapture(video_root)
    while True:
        ret,img = cap.read()
        if ret == True:
            watermark = img_extract_watermark_dct(img,size)
            if len(watermark) != 0:
                break
    cap.release()
    return watermark

In [5]:
def img_embed_watermark_dct(img):
    w, h, _ = img.shape
    base_img = np.zeros((w, h, 3), dtype=np.uint8)
    # make visible watermark image
    visible_watermark = cv2.putText(base_img, visible_text, (10, 30), 0, 1, (255, 255, 255), 4)

    img_ori = img
    beta = round(random.uniform(0.4, 0.6), 4)
    # visible watermark
    img = cv2.addWeighted(src1=img_ori, alpha=1, src2=visible_watermark, beta=beta, gamma=2)
    # blind watermark dct mid-frequency
    text = str(ID)+begin_time
    bitText = toBits(text)
    print("Embed watermark is "+text)
    if ((w / 8) * (h / 8) < len(text)):
        print("Error: Message too large to encode in image")
        return False

    if w % 8 != 0 or h % 8 != 0:
        img = img = cv2.resize(img,(h+(8-h%8),w+(8-w%8)))
    bImg, gImg, rImg = cv2.split(img)
    # message to be hid in blue channel so converted to type float32 for dct function
    bImg = np.float32(bImg)
    # break into 8x8 blocks
    imgBlocks = [np.round(bImg[j:j + 8, i:i + 8] - 128) for (j, i) in itertools.product(range(0, w, 8),
                                                                                        range(0, h, 8))]
    # Blocks are run through DCT function
    dctBlocks = [np.round(cv2.dct(img_Block)) for img_Block in imgBlocks]
    # blocks then run through quantization table
    quantizedDCT = [np.round(dct_Block / quant) for dct_Block in dctBlocks]
    # set LSB in DC value corresponding bit of message
    messIndex = 0
    letterIndex = 0
    for quantizedBlock in quantizedDCT:
        # find LSB in DC coeff and replace with message bit
        DC = quantizedBlock[0][0]
        DC = np.uint8(DC)
        DC = np.unpackbits(DC)
        DC[7] = bitText[messIndex][letterIndex]
        DC = np.packbits(DC)
        DC = np.float32(DC)
        DC = DC - 255
        quantizedBlock[0][0] = DC
        letterIndex = letterIndex + 1
        if letterIndex == 8:
            letterIndex = 0
            messIndex = messIndex + 1
            if messIndex == len(text):
                break
    # blocks run inversely through quantization table
    sImgBlocks = [quantizedBlock * quant + 128 for quantizedBlock in quantizedDCT]
    # blocks run through inverse DCT
    # sImgBlocks = [cv2.idct(B)+128 for B in quantizedDCT]
    # puts the new image back together
    sImg = []
    for chunkRowBlocks in chunks(sImgBlocks, h / 8):
        for rowBlockNum in range(8):
            for block in chunkRowBlocks:
                sImg.extend(block[rowBlockNum])
    sImg = np.array(sImg).reshape(w, h)
    # converted from type float32
    sImg = np.uint8(sImg)
    # show(sImg)
    sImg = cv2.merge((sImg, gImg, rImg))
    return sImg,len(text)

def img_extract_watermark_dct(img,watermark_size):
    w,h,_ = img.shape
    textBits = []
    textsize = None
    buff = 0
    bImg, gImg, rImg = cv2.split(img)
    # message hid in blue channel so converted to type float32 for dct function
    bImg = np.float32(bImg)
    # break into 8x8 blocks
    imgBlocks = [bImg[j:j + 8, i:i + 8] - 128 for (j, i) in itertools.product(range(0, w, 8),
                                                                              range(0, h, 8))]
    # blocks run through quantization table
    quantizedDCT = [img_Block / quant for img_Block in imgBlocks]
    i = 0
    # message extracted from LSB of DC coeff
    for quantizedBlock in quantizedDCT:
        DC = quantizedBlock[0][0]
        DC = np.uint8(DC)
        DC = np.unpackbits(DC)
        if DC[7] == 1:
            buff += (0 & 1) << (7 - i)
        elif DC[7] == 0:
            buff += (1 & 1) << (7 - i)
        i = 1 + i
        if i == 8:
            textBits.append(chr(buff))
            buff = 0
            i = 0
            if textBits[-1] == '*' and textsize is None:
                try:
                    textsize = int(''.join(textBits[:-1]))
                except:
                    pass
        if len(textBits) - len(str(textsize)) - 1 == textsize:
            return ''.join(textBits)[len(str(textsize)) + 1:]
    text = ''
    for i in range(watermark_size):
        text += textBits[i]
    # blocks run inversely through quantization table
    return text
def chunks(l, n):
    m = int(n)
    for i in range(0, len(l), m):
        yield l[i:i + m]

def toBits(text):
    bits = []
    for char in text:
        binval = bin(ord(char))[2:].rjust(8, '0')
        bits.append(binval)
    return bits

def img_embed_watermark_lsb(img,copyright):
    img_copy = img.copy()
    cpr_copy = copyright.copy()

    img_copy = img_copy.astype(np.uint8)
    cpr_copy = cpr_copy.astype(np.uint8)

    cpr_copy[cpr_copy<200] = 1
    cpr_copy[cpr_copy>=200] = 0

    w,h,_ = img_copy.shape

    for i in range(0,w):
        for j in range(0,h):
            img_copy[i,j,:] = (img_copy[i,j,:]//2)*2

    w_c,h_c,_ = cpr_copy.shape
    for i in range(0,w_c):
        for j in range(0,h_c):
            img_copy[i,j,0] = img_copy[i,j,0] + cpr_copy[i,j,0]
    return img_copy

def img_extract_watermark_lsb(img):
    img_copy = img.copy()
    watermark = (img_copy%2)*255
    return watermark

In [6]:
# 生成需要的水印图片
w, h = [240,360]
base_img = np.zeros((w, h, 3), dtype=np.uint8)
# make visible watermark image
visible_watermark = cv2.putText(base_img, visible_text, (10, 30), 0, 1, (255, 255, 255), 4)
visible_watermark = cv2.putText(visible_watermark,"2022",(10,60),0,1,(255,255,255),4)
visible_watermark = cv2.putText(visible_watermark,"USTC",(10,90),0,1,(255,255,255),4)
# cv2.imshow("test",visible_watermark)
# cv2.waitKey()
cv2.imwrite("watermark_pic.jpg",visible_watermark)

True

In [7]:
origin = cv2.imread("test1.jpg")
watermark = cv2.imread("watermark_pic.jpg")
embed_img = img_embed_watermark_lsb(origin,watermark)
# cv2.imshow("embed_img.jpg",embed_img)
cv2.imwrite("embed_img.jpg",embed_img)



True

In [8]:
extract_watermark = img_extract_watermark_lsb(embed_img)
cv2.imshow("extract_img.jpg",extract_watermark)
cv2.waitKey()

-1